# MongoDB

## Conexión con MongoDB y borrado de datos

In [1]:
import pymongo 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import json

from pprintpp import pprint as pp

In [2]:
# connect to database
connection = MongoClient('localhost', 27017)

In [3]:
# Borramos la base de datos
connection.drop_database('black')

# Pymongo permite una sintaxis practicamente igual que la original
db = connection.black

## Lectura en Pandas

Leemos los datos desde el fichero Excel y los desnormalizamos

In [4]:
import pandas as pd
import datetime

In [5]:
df_mov = pd.read_excel("../../data/black.xlsx", sheetname= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheetname= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
id_miembro            7624 non-null int64
fecha                 7624 non-null datetime64[ns]
minuto                7624 non-null int64
hora                  7624 non-null int64
importe               7624 non-null float64
comercio              6889 non-null object
actividad_completa    7623 non-null object
actividad             7623 non-null object
nombre                7624 non-null object
funcion               7624 non-null object
organizacion          6041 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


## Carga en MongoDB

Gracias a Pandas convertimos el dataset en formato JSON, que es el que vamos a insertar en MongoDB prácticamente sin cambios.

In [6]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [7]:
pp(json_list[0])

{
    u'actividad': u'SALUD',
    u'actividad_completa': u'FARMACIAS',
    u'comercio': u'FARMACIA M JESUS MARTINEZ',
    u'fecha': 1252886400000,
    u'funcion': u'concejal',
    u'hora': 9,
    u'id_miembro': 30,
    u'importe': 19.26,
    u'minuto': 52,
    u'nombre': u'Jesús Pedroche Nieto',
    u'organizacion': u'Partido Popular',
}


En MongoDB vamos a crear una única colección donde vamos a guardar todos los datos desnormalizados:

<img src="images/Modelo%20MongoDB.png",width=300,height=100>

<br><br> 

En la inserción de la información en mongo realizan algunas tareas de limpieza:

- Eliminar los campos nulos (no todos)
- Convertir la fecha a un formato DateTime, de forma que mongo inserte el dato correctamente
- Eliminar campos que no queremos tener en el modelo (id_miembro)

In [8]:
for movimiento_json in json_list:
    movimiento_json['fecha'] = datetime.datetime.fromtimestamp(movimiento_json['fecha'] / 1e3)
    
    del movimiento_json['id_miembro']

    if  movimiento_json['comercio'] == None:
        del movimiento_json['comercio']
    
    db.movimientos.insert_one(movimiento_json)

## Querys

Todas las querys de este caso de uso se pueden resolver directamente por la base de datos ...

### Los 10 movimientos mas caros

In [9]:
rs = db.movimientos.find({}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [10]:
pd.DataFrame(list(rs))

,actividad_completa,fecha,importe,nombre
0,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),2007-11-25 01:00:00,11930.00,Ricardo Romero de Tejada y Picatoste
1,VIAJES MARSANS-INTERNACIONAL EXPRESSO,2006-11-01 01:00:00,9825.00,Carlos Vela García
2,EL CORTE INGLES,2009-03-30 02:00:00,9804.15,Ildefonso José Sánchez Barcoj
3,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",2010-08-15 02:00:00,9076.76,Carmen Contreras Gómez
4,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),2006-11-29 01:00:00,8000.00,Enrique de la Torre Martínez
5,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),2005-07-19 02:00:00,7200.00,Ildefonso José Sánchez Barcoj
6,EL CORTE INGLES,2007-08-02 02:00:00,6990.87,Miguel Blesa de la Parra
7,JOYERIAS Y RELOJERIAS,2004-03-30 02:00:00,6905.72,María Elena Gil García
8,JOYERIAS Y RELOJERIAS,2003-08-28 02:00:00,6000.00,María Elena Gil García
9,AGENCIAS DE VIAJES,2004-10-18 02:00:00,6000.00,Mariano Pérez Claver


### Los 10 movimientos mas caros por actividad

In [11]:
rs = db.movimientos.find({
    "actividad": "HOGAR"
}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [12]:
pd.DataFrame(list(rs))

,actividad_completa,fecha,importe,nombre
0,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",2009-05-07 02:00:00,3794.00,Jesús Pedroche Nieto
1,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",2009-07-05 02:00:00,3512.00,Jesús Pedroche Nieto
2,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",2010-02-05 01:00:00,1392.03,Rodrigo de Rato Figaredo
3,MIRO ESTABLECIMIENTOS,2003-08-10 02:00:00,1198.00,Rafael Spottorno Díaz Caro
4,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2009-01-02 01:00:00,1000.41,Jesús Pedroche Nieto
5,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",2005-07-04 02:00:00,998.00,Alejandro Couceiro Ojeda
6,FLORES Y PLANTAS,2007-03-16 01:00:00,864.29,Mariano Pérez Claver
7,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2006-07-31 02:00:00,850.00,María Carmen Cafranga Cavestany
8,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2005-06-15 02:00:00,850.00,Alejandro Couceiro Ojeda
9,"FERRETERIA,BRICOLAJE,MENAJE DEL HOGAR",2006-06-15 02:00:00,660.00,Rafael Darío Fernández Yruegas Moro


### Las 10 personas que mas han gastado

En este caso, al obtener un dato agregado, necesitamos utilizar la función **aggregate()** de mongo

In [13]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$nombre", "importe" : { "$sum" : "$importe"}}},
        {"$sort" : {"importe" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id" : 0, "nombre": "$_id", "importe": 1}}
    ])

In [14]:
df = pd.DataFrame(list(rs))
df

,importe,nombre
0,66444.15,Ildefonso José Sánchez Barcoj
1,40356.29,José Antonio Moral Santín
2,37639.57,Carlos Vela García
3,37608.99,Miguel Blesa de la Parra
4,37472.01,Enrique de la Torre Martínez
5,36590.68,Matías Amat Roca
6,36086.89,Maria Mercedes de la Merced Monge
7,35901.41,Ricardo Romero de Tejada y Picatoste
8,35574.91,Ricardo Morado Iglesias
9,35136.97,Ramón Ferraz Ricarte


### ¿Que comercio es más popular?

Este es un caso especial de agregación, ya que cuenta los distintos. Dentro del flujo de la función se realizan varios pasos para resolver esta query:

- Formar un conjunto con los distintas personas que han comprado en cada comercio
- Desagregar el conjunto
- Agrupar los comercios, contanto los distintos elementos de cada conjunto

In [15]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"},
        {"$group" : { "_id" : "$_id", "cuenta" : { "$sum" : 1}}},
        {"$sort" : {"cuenta" : -1}},
        {"$project" : {"_id" : 0, "comercio": "$_id", "cuenta": 1}},
        {"$match": {"comercio": { "$exists": True, "$ne": None }}},
        {"$limit" : 10}
    ])

In [16]:
df = pd.DataFrame(list(rs))
df

,comercio,cuenta
0,EL CORTE INGLES,47
1,RENFE 001,24
2,IBERIA,22
3,EL CORTE INGLES S.A.,21
4,VIAJES ECI,20
5,REST EL ESPIGON,15
6,FNAC CALLAO,13
7,APARCAMIENTO CORTES,13
8,ESPASA CALPE-CASA DEL LIBRO,12
9,E. S. HIPODROMO,12


Truco ...

En este tipo de querys complicadas podemos ver los resultados parciales ...

In [17]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"}
    ])

pd.DataFrame(list(rs)).head()

,_id,nombres
0,RESTAURANTE LA NICOLETA,Jorge Rábago Juan Aracil
1,RESTAURANTE BICE,Jorge Rábago Juan Aracil
2,AREA DE SERVICIO VILLAMAN,Jorge Rábago Juan Aracil
3,ASADOR LA VEGA,José Caballero Domínguez
4,PIZERIA ARS VIVENDI,José Luis Acero Benedicto
